In [1]:
%reload_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import warnings

import pandas as pd
import numpy as np
import pickle

import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    fbeta_score,
    f1_score,
    make_scorer,
    accuracy_score,
)

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

<IPython.core.display.Javascript object>

In [3]:
def print_vif(x):
    """Utility for checking multicollinearity assumption
    
    :param x: input features to check using VIF. This is assumed to be a pandas.DataFrame
    :return: nothing is returned the VIFs are printed as a pandas series
    """
    # Silence numpy FutureWarning about .ptp
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        x = sm.add_constant(x)

    vifs = []
    for i in range(x.shape[1]):
        vif = variance_inflation_factor(x.values, i)
        vifs.append(vif)

    print("VIF results\n-------------------------------")
    print(pd.Series(vifs, index=x.columns))
    print("-------------------------------\n")

<IPython.core.display.Javascript object>

In [4]:
# To use all
# df_long = pd.read_csv("../data/features_30_sec.csv")
# df_short = pd.read_csv("../data/features_3_sec.csv")
# df = pd.concat((df_long, df_short))

# To use just one
# df = pd.read_csv("../data/features_30_sec.csv")
df = pd.read_csv("../data/features_3_sec.csv")

df["genre"] = df["filename"].str.split(".").str[0]

# "blues.00000.0.wav" -> "blues.00000"
# and
# "blues.00000.wav" -> "blues.00000"
# logic: split on period, take first 2 elements, and but back together
df["songname"] = df["filename"].str.split(".").str[:2].str.join(".")

<IPython.core.display.Javascript object>

In [5]:
m_start = 20  # highest mfcc to use. higher than this is too high in the frequency spectrum to really matter
mel_freq_drops = [f"mfcc{x}_mean" for x in range(m_start, 21)] + [
    f"mfcc{x}_var" for x in range(m_start, 21)
]

<IPython.core.display.Javascript object>

In [6]:
drop_cols = [
    "filename",
    "label",
    "genre",
    "songname",
    "length",
    #     "chroma_stft_mean",
    #     "chroma_stft_var",
    "rms_mean",
    #     "rms_var",
    "spectral_centroid_mean",
    "spectral_centroid_var",
    "spectral_bandwidth_mean",
    "spectral_bandwidth_var",
    "rolloff_mean",
    "rolloff_var",
    #     "zero_crossing_rate_mean",
    #     "zero_crossing_rate_var",
    #     "harmony_mean",
    #     "harmony_var",
    #     "perceptr_mean",
    "perceptr_var",
    #     "tempo",
]

drop_cols = drop_cols + mel_freq_drops
# print_vif(df.drop(drop_cols, 1,))

<IPython.core.display.Javascript object>

In [7]:
# X = df.drop(columns=drop_cols + ["genre"])
X = df.drop(drop_cols, 1)
y = df["genre"]

<IPython.core.display.Javascript object>

In [8]:
X_logged = X.copy()
for c in X_logged:
    if c.endswith("_var"):
        X_logged[c] = np.log(X_logged[c])

<IPython.core.display.Javascript object>

In [9]:
print_vif(X_logged)

VIF results
-------------------------------
const                      1891.875317
chroma_stft_mean              4.034173
chroma_stft_var               2.592091
rms_var                       5.350718
zero_crossing_rate_mean       5.885183
zero_crossing_rate_var        4.828621
harmony_mean                  1.489142
harmony_var                   5.518013
perceptr_mean                 1.590989
tempo                         1.011565
mfcc1_mean                    9.683297
mfcc1_var                     3.725781
mfcc2_mean                    6.195771
mfcc2_var                     2.860859
mfcc3_mean                    2.538644
mfcc3_var                     2.658320
mfcc4_mean                    2.287364
mfcc4_var                     2.904286
mfcc5_mean                    2.794907
mfcc5_var                     2.940476
mfcc6_mean                    3.492312
mfcc6_var                     3.100551
mfcc7_mean                    3.258857
mfcc7_var                     2.875073
mfcc8_mean          

<IPython.core.display.Javascript object>

In [10]:
# og: "blues.00000.0.wav"
# songname: "blues.00000"
# genre: "blues"
song_genre = df[["songname", "genre"]].drop_duplicates()

train_songs, test_songs = train_test_split(
    song_genre["songname"], test_size=0.2, random_state=42, stratify=song_genre["genre"]
)

train_songs = pickle.load(open("../data/train_songs.p", "rb"))
test_songs = pickle.load(open("../data/test_songs.p", "rb"))

train_idxs = df[df["songname"].isin(train_songs)].index
test_idxs = df[df["songname"].isin(test_songs)].index

X_train = X_logged.loc[train_idxs, :]
X_test = X_logged.loc[test_idxs, :]
y_train = y[train_idxs]
y_test = y[test_idxs]

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(7990, 47) (7990,)
(2000, 47) (2000,)


<IPython.core.display.Javascript object>

In [11]:
# Prove no overlap of songs between train/test
set(train_songs).intersection(set(test_songs))

set()

<IPython.core.display.Javascript object>

In [12]:
num_cols = list(X.columns)

bin_cols = []

cat_cols = []
drop_cats = []


preprocessing = ColumnTransformer(
    [
        # Scale numeric columns (not needed for all models but can't hurt)
        ("scaler", StandardScaler(), num_cols)
    ],
    remainder="passthrough",
)


pipeline = Pipeline(
    [
        ("preprocessing", preprocessing),
        #         ("pca", PCA()),
        # Choose your model and put it here
        ("log", LogisticRegression()),
    ]
)


params = {
    "log__penalty": ["l1"],
    "log__C": [0.001, 0.1, 10],
    "log__max_iter": [500],
    # "log__multiclass": ["ovr", "multinomial"],
    # "log__l1_ratio": [0, 0.25, 0.5, 0.75, 1.0],
    "log__solver": ["sag", "saga"],
}


pipeline_cv = GridSearchCV(pipeline, params, verbose=1, n_jobs=-1, cv=2)

pipeline_cv.fit(X_train, y=y_train)


print(pipeline_cv.score(X_train, y_train))
print(pipeline_cv.score(X_test, y_test))
pipeline_cv.best_params_

Fitting 2 folds for each of 6 candidates, totalling 12 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:   11.1s finished


0.7247809762202754
0.6635


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


{'log__C': 0.1,
 'log__max_iter': 500,
 'log__penalty': 'l1',
 'log__solver': 'saga'}

<IPython.core.display.Javascript object>

In [13]:
best = pipeline_cv.best_estimator_.named_steps["log"]
coef_df = pd.DataFrame({"feat": X_train.columns, "coef": best.coef_[0]})
coef_df["abs_coef"] = np.abs(coef_df["coef"])
coef_df.sort_values("abs_coef")

,feat,coef,abs_coef
34,mfcc13_var,0.000000,0.000000
44,mfcc18_var,0.000000,0.000000
42,mfcc17_var,0.000000,0.000000
22,mfcc7_var,0.000000,0.000000
4,zero_crossing_rate_var,0.000000,0.000000
27,mfcc10_mean,0.000000,0.000000
29,mfcc11_mean,0.000000,0.000000
17,mfcc5_mean,0.000000,0.000000
36,mfcc14_var,0.000000,0.000000
10,mfcc1_var,0.000000,0.000000


<IPython.core.display.Javascript object>

In [14]:
add_drop = list(coef_df[coef_df["coef"] == 0]["feat"].values)

<IPython.core.display.Javascript object>

In [15]:
add_drop

['zero_crossing_rate_var',
 'mfcc1_var',
 'mfcc5_mean',
 'mfcc7_var',
 'mfcc10_mean',
 'mfcc11_mean',
 'mfcc11_var',
 'mfcc13_var',
 'mfcc14_mean',
 'mfcc14_var',
 'mfcc17_var',
 'mfcc18_var']

<IPython.core.display.Javascript object>

In [16]:
drop_cols = drop_cols + mel_freq_drops + add_drop
drop_cols

['filename',
 'label',
 'genre',
 'songname',
 'length',
 'rms_mean',
 'spectral_centroid_mean',
 'spectral_centroid_var',
 'spectral_bandwidth_mean',
 'spectral_bandwidth_var',
 'rolloff_mean',
 'rolloff_var',
 'perceptr_var',
 'mfcc20_mean',
 'mfcc20_var',
 'mfcc20_mean',
 'mfcc20_var',
 'zero_crossing_rate_var',
 'mfcc1_var',
 'mfcc5_mean',
 'mfcc7_var',
 'mfcc10_mean',
 'mfcc11_mean',
 'mfcc11_var',
 'mfcc13_var',
 'mfcc14_mean',
 'mfcc14_var',
 'mfcc17_var',
 'mfcc18_var']

<IPython.core.display.Javascript object>

In [17]:
# X = df.drop(columns=drop_cols + ["genre"])
X = df.drop(drop_cols, 1)
y = df["genre"]

<IPython.core.display.Javascript object>

In [18]:
X_logged = X.copy()
for c in X_logged:
    if c.endswith("_var"):
        X_logged[c] = np.log(X_logged[c])

<IPython.core.display.Javascript object>

In [19]:
print_vif(X_logged)

VIF results
-------------------------------
const                      1438.148029
chroma_stft_mean              3.729719
chroma_stft_var               2.485952
rms_var                       4.006543
zero_crossing_rate_mean       4.956602
harmony_mean                  1.478928
harmony_var                   5.191972
perceptr_mean                 1.575567
tempo                         1.009386
mfcc1_mean                    8.808449
mfcc2_mean                    5.755389
mfcc2_var                     2.107250
mfcc3_mean                    2.400324
mfcc3_var                     2.490036
mfcc4_mean                    2.255065
mfcc4_var                     2.851641
mfcc5_var                     2.725289
mfcc6_mean                    3.357283
mfcc6_var                     2.970141
mfcc7_mean                    2.951055
mfcc8_mean                    3.616614
mfcc8_var                     2.520841
mfcc9_mean                    2.682025
mfcc9_var                     2.347265
mfcc10_var          

<IPython.core.display.Javascript object>

In [20]:
# og: "blues.00000.0.wav"
# songname: "blues.00000"
# genre: "blues"
song_genre = df[["songname", "genre"]].drop_duplicates()

train_songs, test_songs = train_test_split(
    song_genre["songname"], test_size=0.2, random_state=42, stratify=song_genre["genre"]
)

train_songs = pickle.load(open("../data/train_songs.p", "rb"))
test_songs = pickle.load(open("../data/test_songs.p", "rb"))

train_idxs = df[df["songname"].isin(train_songs)].index
test_idxs = df[df["songname"].isin(test_songs)].index

X_train = X_logged.loc[train_idxs, :]
X_test = X_logged.loc[test_idxs, :]
y_train = y[train_idxs]
y_test = y[test_idxs]

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(7990, 35) (7990,)
(2000, 35) (2000,)


<IPython.core.display.Javascript object>

In [21]:
num_cols = list(X.columns)

bin_cols = []

cat_cols = []
drop_cats = []


preprocessing = ColumnTransformer(
    [
        # Scale numeric columns (not needed for all models but can't hurt)
        ("scaler", StandardScaler(), num_cols)
    ],
    remainder="passthrough",
)


pipeline = Pipeline(
    [
        ("preprocessing", preprocessing),
        #         ("pca", PCA()),
        # Choose your model and put it here
        ("log", LogisticRegression()),
    ]
)


params = {
    "log__penalty": ["l1"],
    "log__C": [0.001, 0.1, 10],
    "log__max_iter": [500],
    # "log__multiclass": ["ovr", "multinomial"],
    # "log__l1_ratio": [0, 0.25, 0.5, 0.75, 1.0],
    "log__solver": ["sag", "saga"],
}


pipeline_cv = GridSearchCV(pipeline, params, verbose=1, n_jobs=-1, cv=2)

pipeline_cv.fit(X_train, y=y_train)


print(pipeline_cv.score(X_train, y_train))
print(pipeline_cv.score(X_test, y_test))
pipeline_cv.best_params_

Fitting 2 folds for each of 6 candidates, totalling 12 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:    7.4s finished


0.7017521902377972
0.653


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


{'log__C': 0.1,
 'log__max_iter': 500,
 'log__penalty': 'l1',
 'log__solver': 'saga'}

<IPython.core.display.Javascript object>

In [22]:
best = pipeline_cv.best_estimator_.named_steps["log"]
coef_df = pd.DataFrame({"feat": X_train.columns, "coef": best.coef_[0]})
coef_df["abs_coef"] = np.abs(coef_df["coef"])
coef_df.sort_values("abs_coef", ascending=False)

,feat,coef,abs_coef
8,mfcc1_mean,-2.019371,2.019371
9,mfcc2_mean,1.155488,1.155488
5,harmony_var,1.033432,1.033432
2,rms_var,0.953404,0.953404
13,mfcc4_mean,0.874160,0.874160
16,mfcc6_mean,0.853201,0.853201
10,mfcc2_var,-0.717932,0.717932
3,zero_crossing_rate_mean,0.672025,0.672025
18,mfcc7_mean,-0.498572,0.498572
15,mfcc5_var,0.447887,0.447887


<IPython.core.display.Javascript object>

In [28]:
X_train.columns

Index(['chroma_stft_mean', 'chroma_stft_var', 'rms_var',
       'zero_crossing_rate_mean', 'harmony_mean', 'harmony_var',
       'perceptr_mean', 'tempo', 'mfcc1_mean', 'mfcc2_mean', 'mfcc2_var',
       'mfcc3_mean', 'mfcc3_var', 'mfcc4_mean', 'mfcc4_var', 'mfcc5_var',
       'mfcc6_mean', 'mfcc6_var', 'mfcc7_mean', 'mfcc8_mean', 'mfcc8_var',
       'mfcc9_mean', 'mfcc9_var', 'mfcc10_var', 'mfcc12_mean', 'mfcc12_var',
       'mfcc13_mean', 'mfcc15_mean', 'mfcc15_var', 'mfcc16_mean', 'mfcc16_var',
       'mfcc17_mean', 'mfcc18_mean', 'mfcc19_mean', 'mfcc19_var'],
      dtype='object')

<IPython.core.display.Javascript object>

In [23]:
keep_cols = coef_df.sort_values("abs_coef", ascending=False)["feat"][:10]

<IPython.core.display.Javascript object>

In [24]:
value_cast = {
    "metal": 1,
    "jazz": 2,
    "blues": 3,
    "reggae": 4,
    "pop": 5,
    "disco": 6,
    "classical": 7,
    "hiphop": 8,
    "rock": 9,
    "country": 0,
}

logit_y_train = y_train.replace(value_cast)

<IPython.core.display.Javascript object>

In [25]:
from sklearn.feature_selection import (
    SelectKBest,
    f_classif,
    f_regression,
    mutual_info_regression,
)

<IPython.core.display.Javascript object>

In [26]:
selector = SelectKBest()
selector.fit(X_train, y_train)

k_best = selector.transform(X_train)

<IPython.core.display.Javascript object>

In [27]:
# We can see/rank which features were the best
score_df = pd.DataFrame({"feature": X_train.columns, "f_score": selector.scores_})
score_df = score_df.sort_values("f_score", ascending=False)
print(score_df.head(40))

# We can put back into a dataframe to see column names
best_df = pd.DataFrame(k_best, columns=X_train.columns[selector.get_support()])
best_df.head()

                    feature     f_score
0          chroma_stft_mean  858.813725
8                mfcc1_mean  811.704964
2                   rms_var  788.759820
13               mfcc4_mean  550.663118
9                mfcc2_mean  541.594843
14                mfcc4_var  507.873557
15                mfcc5_var  460.828668
17                mfcc6_var  453.393656
19               mfcc8_mean  423.369217
16               mfcc6_mean  408.250749
5               harmony_var  403.504903
12                mfcc3_var  358.995792
21               mfcc9_mean  354.050426
3   zero_crossing_rate_mean  313.514158
18               mfcc7_mean  311.531293
26              mfcc13_mean  288.663688
24              mfcc12_mean  276.866465
10                mfcc2_var  270.397873
11               mfcc3_mean  266.872376
20                mfcc8_var  265.610448
31              mfcc17_mean  261.580310
1           chroma_stft_var  244.581605
27              mfcc15_mean  238.874546
23               mfcc10_var  204.727636


,chroma_stft_mean,rms_var,mfcc1_mean,mfcc2_mean,mfcc4_mean,mfcc4_var,mfcc5_var,mfcc6_mean,mfcc6_var,mfcc8_mean
0,0.335406,-5.649009,-118.627914,125.083626,41.321484,5.202329,5.030197,20.115141,4.326148,17.855198
1,0.343065,-6.536409,-125.590706,122.421227,50.128387,4.960747,4.936221,21.385401,4.354372,19.454103
2,0.346815,-5.377274,-132.441940,115.085175,50.189293,4.970966,4.854970,24.650375,4.195263,15.643386
3,0.363639,-6.012662,-118.231087,132.116501,39.769306,5.218697,4.972575,20.468134,4.808245,18.745104
4,0.335579,-6.376606,-105.968376,134.643646,40.171753,4.635900,4.631337,18.734617,4.370334,19.207966


<IPython.core.display.Javascript object>